## Determine UM domain height

In [1]:
from pathlib import Path

import iris
import numpy as np

In [2]:
from aeolus.const import init_const

In [3]:
from util_mypaths import path_to_constants, path_to_data_umserve

In [4]:
# Functions to estimate the required model domain height
def calc_top_of_atm_height(planet_specific_gas_constant, planet_surface_gravity):
    """Calculate the top of the atmosphere height in [m]."""
    r_s = planet_specific_gas_constant.data
    g = planet_surface_gravity.data
    pres_0 = 2e7  # [Pa]
    pres = 1  # [Pa]
    temp = 950  # [K] adjust based on the P-T profile from ATMO !!!

    hght = -(r_s * temp / g) * np.log(pres / pres_0)
    return hght


def calc_top_of_atm_pressure(planet_specific_gas_constant, planet_surface_gravity):
    """Calculate the top of the atmosphere pressure in [Pa]."""
    r_s = planet_specific_gas_constant.data
    g = planet_surface_gravity.data
    pres_0 = 2e7  # [Pa]
    temp = 950  # [K] adjust based on the P-T profile from ATMO !!!
    hght = 4.6e6  # [m] adjust

    pres = pres_0 * np.exp(-hght * g / (r_s * temp))
    return pres


def exner_to_pressure(exner, planet_specific_gas_constant, planet_specific_heat_capacity):
    """Convert the non-dimentional (Exner) pressure to pressure in [Pa]."""
    r_s = planet_specific_gas_constant.data
    c_p = planet_specific_heat_capacity.data
    pres_0 = 2e7  # [Pa]

    pres = pres_0 * exner ** (c_p / r_s)
    return pres

#### Find theoretical top-of-the-atmosphere height and pressure

In [5]:
hatp11b_const = init_const("hatp11b", path_to_constants)
hd189733b_const = init_const("hd189733b", path_to_constants)
hd209458b_const = init_const("hd209458b", path_to_constants)
wasp17b_const = init_const("wasp17b", path_to_constants)

planet_const = hatp11b_const

print(
    "TOA height [m]:",
    calc_top_of_atm_height(
        planet_const.planet_specific_gas_constant, planet_const.planet_surface_gravity
    ),
)
print(
    "TOA pressure [Pa]:",
    calc_top_of_atm_pressure(
        planet_const.planet_specific_gas_constant, planet_const.planet_surface_gravity
    ),
)

TOA height [m]: 4251901.088635854
TOA pressure [Pa]: 0.25250613767160984


#### Check top-of-the-atmosphere pressure in the UM startdump

In [6]:
planet = "hatp11b"
path_to_astart = path_to_data_umserve / "um_runs" / planet / "equilibrium" / "u-bl336"
fname = "atmos.astart_4.6e6_1.60e+00"
exner = iris.load_cube(str(path_to_astart / fname), "dimensionless_exner_function")

/home/mz355/miniconda3/lib/python3.8/site-packages/iris/fileformats/_ff.py:739: UserWarning: The stash code m01s00i060 is on a grid 22 which has not been explicitly handled by the fieldsfile loader. Assuming the data is on a P grid.
  warnings.warn(
/home/mz355/miniconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/mz355/miniconda3/lib/python3.8/site-packages/numpy/core/_methods.py:188: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)


In [7]:
print(
    f"Min pressure [Pa]: {exner_to_pressure(exner, planet_const.planet_specific_gas_constant, planet_const.planet_specific_heat_capacity).data.min():0.2e}"
)
print(
    f"Max pressure [Pa]: {exner_to_pressure(exner, planet_const.planet_specific_gas_constant, planet_const.planet_specific_heat_capacity).data.max():0.2e}"
)

Min pressure [Pa]: 1.60e+00
Max pressure [Pa]: 1.82e+07


In [8]:
# Checks for HAT-P-11b
# atmos.astart_<height>_<pressure>
# atmos.astart_3.8e6_6.67e+01
# atmos.astart_4.1e6_1.68e+01
# atmos.astart_4.3e6_6.69e+00
# atmos.astart_4.4e6_4.09e+00
# atmos.astart_4.6e6_1.60e+00